# Table 4.   Factors associated with reduction in VAS, Multivariate Analysis

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Liste Notärzte-1.xlsx'

In [ ]:
restrict_to_trauma = True
restrict_to_primary = True

In [ ]:
# Load and preprocess data
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)

# Merge with physician data
medic_df = medic_df.drop_duplicates('Mitglieder mit Einsatzfunktion')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

# Remove duplicates
data_df = data_df.drop_duplicates(subset=["SNZ Ereignis Nr. "])

# Filter patients with VAS > 3 at scene
data_df = data_df[data_df["VAS_on_scene"] > 3]

print(f"Total patients after filtering: {len(data_df)}")
print(f"Adult patients: {(data_df['Alter '] >= 16).sum()}")
print(f"Pediatric patients: {(data_df['Alter '] < 16).sum()}")

In [ ]:
if restrict_to_trauma:
    n_non_trauma = data_df[data_df['Einteilung (reduziert)'] != 'Unfall'].shape[0]
    print(f'Excluded {n_non_trauma} non-trauma patients')

    # adult non-trauma patients
    n_adult_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] >= 16)].shape[0]
    print(f'Excluded {n_adult_non_trauma} adult non-trauma patients')
    # pediatric non-trauma patients
    n_pediatric_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] < 16)].shape[0]
    print(f'Excluded {n_pediatric_non_trauma} pediatric non-trauma patients')

    data_df = data_df[data_df['Einteilung (reduziert)'] == 'Unfall']

In [ ]:
if restrict_to_primary:
    n_secondary = data_df[data_df['Einsatzart'] != 'Primär'].shape[0]
    print(f'Excluded {n_secondary} secondary transport patients')

    # adult secondary transport patients
    n_adult_secondary = data_df[(data_df['Einsatzart'] != 'Primär') & (data_df["Alter "] >= 16)].shape[0]
    print(f'Excluded {n_adult_secondary} adult secondary transport patients')
    # pediatric secondary transport patients
    n_pediatric_secondary = data_df[(data_df['Einsatzart'] != 'Primär') & (data_df["Alter "] < 16)].shape[0]
    print(f'Excluded {n_pediatric_secondary} pediatric secondary transport patients')
    data_df = data_df[data_df['Einsatzart'] == 'Primär']

# Adult Patients (≥16 years)

In [ ]:
# Prepare adult-only dataset (≥16 years)
adult_df = data_df[data_df['Alter '] >= 16].copy()

# Create outcome variable: VAS reduction (scene to arrival)
adult_df['vas_reduction'] = adult_df['VAS_on_scene'] - adult_df['VAS_on_arrival']

# Create predictor variables for adult patients
adult_df['age'] = adult_df['Alter ']
adult_df['male_patient'] = (adult_df['Geschlecht'] == 'Männlich').astype(int)
adult_df['male_physician'] = (adult_df['physician_sex'] == 'm').astype(int)
adult_df['primary_mission'] = (adult_df['Einsatzart'] == 'Primär').astype(int)
adult_df['night_mission'] = (adult_df['Tag oder Nacht'] == 'Nacht').astype(int)
adult_df['winter_season'] = np.where(adult_df['Monat'].isin(['Oktober', 'November', 'Dezember', 'Januar', 'Februar', 'März']), 1, 0).astype(int)
adult_df['trauma'] = adult_df['Einteilung (reduziert)'].str.contains('Unfall', na=False).astype(int)
adult_df['winch_extraction'] = adult_df['Bergungen'].str.contains('Winde', na=False).astype(int)
adult_df['vas_scene'] = adult_df['VAS_on_scene']
adult_df['mission_duration'] = (
    pd.to_datetime(adult_df['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - 
    pd.to_datetime(adult_df['Erstbefund'], format='%d.%m.%Y %H:%M:%S')
).dt.total_seconds() / 60

# Create medication dose variables (matching Table 1 approach)
adult_df['fentanyl_dose'] = 0
adult_df['ketamine_dose'] = 0
adult_df['esketamine_dose'] = 0
adult_df['morphine_dose'] = 0
adult_df['Alle Medikamente'] = adult_df['Alle Medikamente'].str.replace(',', ';')  # replace commas with semicolons for consistency
for i, row in adult_df.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        # remove mcg or mg from dose
        if '7IE' in analgetic:
                print(f"Skipping dose with 7IE: {analgetic}")
                continue

        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        if 'Fentanyl' in analgetic and '/h' not in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip()
            adult_df.at[i, 'fentanyl_dose'] += float(dose) 
        elif 'Fentanyl' in analgetic and '/h' in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
            dose = float(dose) * adult_df.at[i, 'mission_duration']  
            adult_df.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
            dose = analgetic.split('Ketamin')[-1].strip()
            adult_df.at[i, 'ketamine_dose'] += float(dose)
        elif 'Esketamin' in analgetic:
            dose = analgetic.split('Esketamin')[-1].strip()
            adult_df.at[i, 'esketamine_dose'] += float(dose)
        elif 'Morphin' in analgetic or 'Morphine' in analgetic:
            dose = analgetic.split('Morphin')[-1].strip()
            adult_df.at[i, 'morphine_dose'] += float(dose)

# Create medication variables
adult_df['fentanyl_given'] = adult_df['fentanyl_dose'] > 0
adult_df['morphine_given'] = adult_df['morphine_dose'] > 0
adult_df['ketamine_given'] = adult_df['ketamine_dose'] > 0
adult_df['esketamine_given'] = adult_df['esketamine_dose'] > 0

# Create combined medication variables (PRIMARY VARIABLES OF INTEREST)
adult_df['any_opiate_dose'] = adult_df['morphine_dose'] + adult_df['fentanyl_dose']
adult_df['any_ketamine_dose'] = adult_df['ketamine_dose'] + adult_df['esketamine_dose']
adult_df['any_opiate_given'] = (adult_df['morphine_dose'] > 0) | (adult_df['fentanyl_dose'] > 0)
adult_df['any_ketamine_given'] = (adult_df['ketamine_dose'] > 0) | (adult_df['esketamine_dose'] > 0)

# Additional medication variables
adult_df['opiate_ketamine_combination'] = (adult_df['any_opiate_given'] & adult_df['any_ketamine_given']).astype(int)
adult_df['no_analgesic'] = ((adult_df['any_opiate_given'] == 0) & (adult_df['any_ketamine_given'] == 0)).astype(int)

# Decompose patient / doctor sex into combinations
adult_df['dr_male_pt_male'] = adult_df['male_physician'] & adult_df['male_patient']
adult_df['dr_female_pt_female'] = ((adult_df['male_physician'] == 0) & (adult_df['male_patient'] == 0)).astype(int)
adult_df['dr_male_pt_female'] = (adult_df['male_physician'] & (adult_df['male_patient'] == 0)).astype(int)
adult_df['dr_female_pt_male'] = ((adult_df['male_physician'] == 0) & adult_df['male_patient']).astype(int)

# Variables for VAS reduction analysis
adult_model_vars = ['vas_reduction', 'any_opiate_dose', 'any_ketamine_dose', 'age', 'NACA',
                    # 'dr_male_pt_male', 'dr_female_pt_female', 'dr_male_pt_female', 'dr_female_pt_male',
                    'male_physician', 'male_patient',
                    'mission_duration', 'primary_mission', 'night_mission', 'winter_season',
                    'winch_extraction', 'vas_scene', 'opiate_ketamine_combination', 'no_analgesic']

# print number of na per variable in adult_model_vars
for var in adult_model_vars:
    n_na = adult_df[var].isna().sum()
    print(f"{var}: {n_na} NA values")

adult_df_clean = adult_df[adult_model_vars].dropna()

print(f"Adult patients included in VAS reduction analysis: {len(adult_df_clean)}")
print(f"Mean VAS reduction: {adult_df_clean['vas_reduction'].mean():.2f} ± {adult_df_clean['vas_reduction'].std():.2f}")
print(f"Median VAS reduction: {adult_df_clean['vas_reduction'].median():.2f}")
print(f"VAS reduction range: {adult_df_clean['vas_reduction'].min():.1f} to {adult_df_clean['vas_reduction'].max():.1f}")

In [ ]:
# Linear regression analysis functions for VAS reduction
def univariate_linear_regression(df, outcome_var, predictor_vars):
    """
    Perform univariate linear regression for each predictor variable
    """
    results = []
    
    for var in predictor_vars:
        # Prepare data
        X = df[[var]].copy()
        y = df[outcome_var]
        
        # Add constant for intercept
        X_with_const = sm.add_constant(X)
        
        # Fit linear regression
        try:
            model = sm.OLS(y, X_with_const).fit()
            
            # Extract results
            coef = model.params[var]
            se = model.bse[var]
            ci_lower = model.conf_int().loc[var, 0]
            ci_upper = model.conf_int().loc[var, 1]
            p_value = model.pvalues[var]
            r_squared = model.rsquared
            
            results.append({
                'Variable': var,
                'Coefficient': coef,
                'SE': se,
                'CI_lower': ci_lower,
                'CI_upper': ci_upper,
                'P_value': p_value,
                'R_squared': r_squared,
                'Coef_CI': f"{coef:.3f} ({ci_lower:.3f} to {ci_upper:.3f})",
                'P_formatted': f"{p_value:.3f}" if p_value >= 0.001 else "<0.001"
            })
            
        except Exception as e:
            print(f"Error with variable {var}: {e}")
            
    return pd.DataFrame(results)

def multivariate_linear_regression(df, outcome_var, predictor_vars):
    """
    Perform multivariate linear regression
    """
    # Prepare data
    X = df[predictor_vars].copy()
    y = df[outcome_var]
    
    # Add constant for intercept
    X_with_const = sm.add_constant(X)
    
    # Fit linear regression
    model = sm.OLS(y, X_with_const).fit()
    
    # Extract results
    results = []
    for var in predictor_vars:
        coef = model.params[var]
        se = model.bse[var]
        ci_lower = model.conf_int().loc[var, 0]
        ci_upper = model.conf_int().loc[var, 1]
        p_value = model.pvalues[var]
        
        results.append({
            'Variable': var,
            'Coefficient': coef,
            'SE': se,
            'CI_lower': ci_lower,
            'CI_upper': ci_upper,
            'P_value': p_value,
            'Coef_CI': f"{coef:.3f} ({ci_lower:.3f} to {ci_upper:.3f})",
            'P_formatted': f"{p_value:.3f}" if p_value >= 0.001 else "<0.001"
        })
    
    results_df = pd.DataFrame(results)
    
    # Add model statistics
    model_stats = {
        'R_squared': model.rsquared,
        'Adj_R_squared': model.rsquared_adj,
        'F_statistic': model.fvalue,
        'F_pvalue': model.f_pvalue,
        'AIC': model.aic,
        'BIC': model.bic,
        'MSE': model.mse_resid
    }
    
    return results_df, model_stats, model

In [ ]:
# Perform VAS reduction analysis
predictor_vars = ['any_opiate_dose', 'any_ketamine_dose', 'age', 'NACA',
                #   'dr_male_pt_male', 'dr_male_pt_female', 'dr_female_pt_male',
                                    'male_physician', 'male_patient',
                  'mission_duration', 'night_mission', 'winter_season',
                  'winch_extraction', 'vas_scene']

# Univariate analysis for VAS reduction
print("UNIVARIATE LINEAR REGRESSION ANALYSIS - VAS REDUCTION")
print("=" * 60)
vas_univariate_results = univariate_linear_regression(adult_df_clean, 'vas_reduction', predictor_vars)

for _, row in vas_univariate_results.iterrows():
    print(f"{row['Variable']:<30}: β = {row['Coef_CI']:<25} p = {row['P_formatted']:<8} R² = {row['R_squared']:.3f}")

# Multivariate analysis for VAS reduction
print(f"\nMULTIVARIATE LINEAR REGRESSION ANALYSIS - VAS REDUCTION")
print("=" * 60)
vas_multivariate_results, vas_model_stats, vas_fitted_model = multivariate_linear_regression(
    adult_df_clean, 'vas_reduction', predictor_vars)

for _, row in vas_multivariate_results.iterrows():
    print(f"{row['Variable']:<30}: β = {row['Coef_CI']:<25} p = {row['P_formatted']}")

print(f"\nModel Statistics:")
print(f"R-squared: {vas_model_stats['R_squared']:.3f}")
print(f"Adjusted R-squared: {vas_model_stats['Adj_R_squared']:.3f}")
print(f"F-statistic: {vas_model_stats['F_statistic']:.2f}")
print(f"F p-value: {vas_model_stats['F_pvalue']:.3f}")
print(f"Sample size: {len(adult_df_clean)} patients")

In [ ]:
# Create formatted Table 4: Factors Associated with VAS Reduction
def create_table4(univariate_df, multivariate_df):
    """
    Create a publication-ready Table 4 for VAS reduction analysis
    """
    # Define variable labels for better presentation
    variable_labels = {
        'any_opiate_dose': 'Any opiate dose (mcg)',
        'any_ketamine_dose': 'Any ketamine dose (mg)',
        'age': 'Age (years)',
        'NACA': 'NACA score',
        # 'dr_male_pt_male': 'Male doctor - Male patient',
        # 'dr_male_pt_female': 'Male doctor - Female patient', 
        # 'dr_female_pt_male': 'Female doctor - Male patient',
        'male_physician': 'Male physician',
        'male_patient': 'Male patient',
        'mission_duration': 'Mission duration (minutes)',
        'primary_mission': 'Primary mission',
        'night_mission': 'Night mission',
        'winter_season': 'Winter season',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'VAS at scene',
    }
    
    # Create combined table
    table4_data = []
    
    # Merge univariate and multivariate results
    for var in univariate_df['Variable']:
        uni_row = univariate_df[univariate_df['Variable'] == var].iloc[0]
        
        # Check if variable is in multivariate model
        multi_row = multivariate_df[multivariate_df['Variable'] == var]
        
        if len(multi_row) > 0:
            multi_row = multi_row.iloc[0]
            multi_coef_ci = multi_row['Coef_CI']
            multi_p = multi_row['P_formatted']
        else:
            multi_coef_ci = '-'
            multi_p = '-'
        
        table4_data.append({
            'Variable': variable_labels.get(var, var),
            'Univariate_Coef_CI': uni_row['Coef_CI'],
            'Univariate_P': uni_row['P_formatted'],
            'Multivariate_Coef_CI': multi_coef_ci,
            'Multivariate_P': multi_p
        })
    
    table4_df = pd.DataFrame(table4_data)
    return table4_df

# Create Table 4
vas_table4 = create_table4(vas_univariate_results, vas_multivariate_results)

print("Table 4. Factors Associated with VAS Reduction - ADULT PATIENTS (≥16 years)")
print("=" * 90)
print(f"{'Variable':<35} {'Univariate':<25} {'P-value':<10} {'Multivariate':<25} {'P-value':<10}")
print(f"{'':35} {'β (95% CI)':<25} {'':10} {'β (95% CI)':<25} {'':10}")
print("-" * 90)

for _, row in vas_table4.iterrows():
    print(f"{row['Variable']:<35} {row['Univariate_Coef_CI']:<25} {row['Univariate_P']:<10} {row['Multivariate_Coef_CI']:<25} {row['Multivariate_P']:<10}")

print("\nβ = Beta coefficient (change in VAS reduction per unit increase in predictor)")
print("CI = Confidence Interval")
print(f"Model includes {len(adult_df_clean)} adult patients")
print(f"Mean VAS reduction: {adult_df_clean['vas_reduction'].mean():.2f} (SD: {adult_df_clean['vas_reduction'].std():.2f})")
print(f"Model R-squared: {vas_model_stats['R_squared']:.3f}")

# Show significant variables
vas_significant_vars = vas_multivariate_results[vas_multivariate_results['P_value'] < 0.05]
if len(vas_significant_vars) > 0:
    print(f"\nSignificant Variables in Multivariate Model (p < 0.05):")
    for _, row in vas_significant_vars.iterrows():
        direction = "increases" if row['Coefficient'] > 0 else "decreases"
        print(f"- {row['Variable']}: β = {row['Coefficient']:.3f}, {direction} VAS reduction")
else:
    print("\nNo variables reached statistical significance (p < 0.05)")

In [ ]:
# save vas_table4
# vas_table4.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/analysis/adult_trauma/table4_vas_reduction.csv', index=False)

# Summary Figures for VAS Reduction Analysis

In [ ]:
# Dose-Response Analysis
def analyze_dose_response(df):
    """
    Analyze dose-response relationships for opiates and ketamine
    """
    # Create dose categories for better visualization
    df_analysis = df.copy()
    
    # Opiate dose categories
    df_analysis['opiate_dose_cat'] = pd.cut(df_analysis['any_opiate_dose'], 
                                          bins=[0, 0.1, 50, 100, 200, float('inf')],
                                          labels=['None', '≤50mcg', '51-100mcg', '101-200mcg', '>200mcg'],
                                          include_lowest=True)
    
    # Ketamine dose categories  
    df_analysis['ketamine_dose_cat'] = pd.cut(df_analysis['any_ketamine_dose'],
                                            bins=[0, 0.1, 25, 50, 100, float('inf')],
                                            labels=['None', '≤25mg', '26-50mg', '51-100mg', '>100mg'],
                                            include_lowest=True)
    
    # Calculate mean VAS reduction by dose category
    opiate_dose_response = df_analysis.groupby('opiate_dose_cat')['vas_reduction'].agg([
        'count', 'mean', 'std', 'sem'
    ]).reset_index()
    
    ketamine_dose_response = df_analysis.groupby('ketamine_dose_cat')['vas_reduction'].agg([
        'count', 'mean', 'std', 'sem'
    ]).reset_index()
    
    return df_analysis, opiate_dose_response, ketamine_dose_response

# Perform dose-response analysis
df_dose_analysis, opiate_response, ketamine_response = analyze_dose_response(adult_df_clean)

print("DOSE-RESPONSE ANALYSIS")
print("=" * 50)
print("\nOpiate Dose Response:")
print(opiate_response)
print("\nKetamine Dose Response:")
print(ketamine_response)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Set style for publication-quality figures
plt.style.use('default')
sns.set_palette("husl")

# Create comprehensive summary figure for VAS reduction analysis
def create_vas_reduction_summary_figure():
    """
    Create a comprehensive summary figure showing VAS reduction analysis
    """
    fig = plt.figure(figsize=(20, 14))
    gs = fig.add_gridspec(3, 4, hspace=0.4, wspace=0.4, height_ratios=[1, 1, 1.3])
    
    # 1. VAS Reduction Distribution
    ax1 = fig.add_subplot(gs[0, 0])
    adult_df_clean['vas_reduction'].hist(bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    ax1.axvline(adult_df_clean['vas_reduction'].mean(), color='red', linestyle='--', 
                label=f'Mean: {adult_df_clean["vas_reduction"].mean():.2f}')
    ax1.set_xlabel('VAS Reduction (Scene - Arrival)', fontweight='bold')
    ax1.set_ylabel('Frequency', fontweight='bold')
    ax1.set_title('Distribution of VAS Reduction', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Opiate Dose vs VAS Reduction (Scatter with CI)
    ax2 = fig.add_subplot(gs[0, 1])
    # Filter out zero doses for better visualization
    opiate_data = adult_df_clean[adult_df_clean['any_opiate_dose'] > 0]
    if len(opiate_data) > 0:
        # Add jitter to x-axis to reduce overplotting
        jitter_strength = 0.02 * (opiate_data['any_opiate_dose'].max() - opiate_data['any_opiate_dose'].min())
        x_jittered = opiate_data['any_opiate_dose'] + np.random.normal(0, jitter_strength, len(opiate_data))
        
        ax2.scatter(x_jittered, opiate_data['vas_reduction'], 
                   alpha=0.3, color='orange', s=20)
        
        # Add trend line with 95% CI
        if len(opiate_data) > 2:
            x_vals = np.linspace(opiate_data['any_opiate_dose'].min(), 
                               opiate_data['any_opiate_dose'].max(), 100)
            
            # Calculate regression and confidence interval
            slope, intercept, r_value, p_value, std_err = stats.linregress(
                opiate_data['any_opiate_dose'], opiate_data['vas_reduction'])
            
            y_vals = slope * x_vals + intercept
            
            # Calculate 95% CI for the regression line
            n = len(opiate_data)
            x_mean = opiate_data['any_opiate_dose'].mean()
            ssx = np.sum((opiate_data['any_opiate_dose'] - x_mean) ** 2)
            se_fit = np.sqrt(std_err**2 * n * (1/n + (x_vals - x_mean)**2 / ssx))
            ci = 1.96 * se_fit  # 95% CI
            
            ax2.plot(x_vals, y_vals, "r-", alpha=0.8, linewidth=2, label='Trend line')
            ax2.fill_between(x_vals, y_vals - ci, y_vals + ci, 
                           color='red', alpha=0.2, label='95% CI')
            ax2.legend(fontsize=8)
    
    ax2.set_xlabel('Opiate Dose (mcg)', fontweight='bold')
    ax2.set_ylabel('VAS Reduction', fontweight='bold')
    ax2.set_title('Opiate Dose vs VAS Reduction', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # 3. Ketamine Dose vs VAS Reduction (Scatter with CI)
    ax3 = fig.add_subplot(gs[0, 2])
    ketamine_data = adult_df_clean[adult_df_clean['any_ketamine_dose'] > 0]
    if len(ketamine_data) > 0:
        # Add jitter to x-axis to reduce overplotting
        jitter_strength = 0.02 * (ketamine_data['any_ketamine_dose'].max() - ketamine_data['any_ketamine_dose'].min())
        x_jittered = ketamine_data['any_ketamine_dose'] + np.random.normal(0, jitter_strength, len(ketamine_data))
        
        ax3.scatter(x_jittered, ketamine_data['vas_reduction'], 
                   alpha=0.3, color='green', s=20)
        
        # Add trend line with 95% CI
        if len(ketamine_data) > 2:
            x_vals = np.linspace(ketamine_data['any_ketamine_dose'].min(), 
                               ketamine_data['any_ketamine_dose'].max(), 100)
            
            # Calculate regression and confidence interval
            slope, intercept, r_value, p_value, std_err = stats.linregress(
                ketamine_data['any_ketamine_dose'], ketamine_data['vas_reduction'])
            
            y_vals = slope * x_vals + intercept
            
            # Calculate 95% CI for the regression line
            n = len(ketamine_data)
            x_mean = ketamine_data['any_ketamine_dose'].mean()
            ssx = np.sum((ketamine_data['any_ketamine_dose'] - x_mean) ** 2)
            se_fit = np.sqrt(std_err**2 * n * (1/n + (x_vals - x_mean)**2 / ssx))
            ci = 1.96 * se_fit  # 95% CI
            
            ax3.plot(x_vals, y_vals, "r-", alpha=0.8, linewidth=2, label='Trend line')
            ax3.fill_between(x_vals, y_vals - ci, y_vals + ci, 
                           color='red', alpha=0.2, label='95% CI')
            ax3.legend(fontsize=8)
    
    ax3.set_xlabel('Ketamine Dose (mg)', fontweight='bold')
    ax3.set_ylabel('VAS Reduction', fontweight='bold')
    ax3.set_title('Ketamine Dose vs VAS Reduction', fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # 4. Combination Therapy vs Monotherapy
    ax4 = fig.add_subplot(gs[0, 3])
    therapy_data = []
    therapy_labels = []
    
    # No analgesic
    no_analgesic = adult_df_clean[adult_df_clean['no_analgesic'] == 1]['vas_reduction']
    therapy_data.append(no_analgesic)
    therapy_labels.append(f'No\nAnalgesic\n(n={len(no_analgesic)})')
    
    # Opiate only (using dose variables instead of given variables)
    opiate_only = adult_df_clean[(adult_df_clean['any_opiate_dose'] > 0) & 
                                (adult_df_clean['any_ketamine_dose'] == 0)]['vas_reduction']
    therapy_data.append(opiate_only)
    therapy_labels.append(f'Opiate\nOnly\n(n={len(opiate_only)})')
    
    # Ketamine only (using dose variables instead of given variables)
    ketamine_only = adult_df_clean[(adult_df_clean['any_opiate_dose'] == 0) & 
                                  (adult_df_clean['any_ketamine_dose'] > 0)]['vas_reduction']
    therapy_data.append(ketamine_only)
    therapy_labels.append(f'Ketamine\nOnly\n(n={len(ketamine_only)})')
    
    # Combination
    combination = adult_df_clean[adult_df_clean['opiate_ketamine_combination'] == 1]['vas_reduction']
    therapy_data.append(combination)
    therapy_labels.append(f'Combination\n(n={len(combination)})')
    
    bp = ax4.boxplot(therapy_data, labels=therapy_labels, patch_artist=True)
    colors = ['lightcoral', 'deepskyblue', 'orange', 'mediumseagreen']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)

    ax4.set_ylabel('VAS Reduction', fontweight='bold')
    ax4.set_title('VAS Reduction by Therapy Type', fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='y')
    
    # 5. Dose-Response: Opiate Categories
    ax5 = fig.add_subplot(gs[1, 0:2])
    opiate_response_filtered = opiate_response.dropna()
    if len(opiate_response_filtered) > 0:
        bars = ax5.bar(range(len(opiate_response_filtered)), opiate_response_filtered['mean'], 
                      yerr=opiate_response_filtered['sem'], capsize=5, 
                      color='deepskyblue', alpha=0.7, edgecolor='black')
        ax5.set_xticks(range(len(opiate_response_filtered)))
        ax5.set_xticklabels(opiate_response_filtered['opiate_dose_cat'], rotation=0, ha='center')
        ax5.set_ylabel('Mean VAS Reduction ± SEM', fontweight='bold')
        ax5.set_title('Dose-Response: Opiate Categories', fontweight='bold')
        ax5.grid(True, alpha=0.3, axis='y')
        
        # Add sample sizes on bars
        for i, (bar, count) in enumerate(zip(bars, opiate_response_filtered['count'])):
            # ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
            ax5.text(bar.get_x() + bar.get_width()/2, 0.1, 
                    f'n={count}', ha='center', va='bottom', fontsize=9)
    
    # 6. Dose-Response: Ketamine Categories
    ax6 = fig.add_subplot(gs[1, 2:4])
    ketamine_response_filtered = ketamine_response.dropna()
    if len(ketamine_response_filtered) > 0:
        bars = ax6.bar(range(len(ketamine_response_filtered)), ketamine_response_filtered['mean'], 
                      yerr=ketamine_response_filtered['sem'], capsize=5, 
                      color='orange', alpha=0.7, edgecolor='black')
        ax6.set_xticks(range(len(ketamine_response_filtered)))
        ax6.set_xticklabels(ketamine_response_filtered['ketamine_dose_cat'], rotation=0, ha='center')
        ax6.set_ylabel('Mean VAS Reduction ± SEM', fontweight='bold')
        ax6.set_title('Dose-Response: Ketamine Categories', fontweight='bold')
        ax6.grid(True, alpha=0.3, axis='y')
        
        # Add sample sizes on bars
        for i, (bar, count) in enumerate(zip(bars, ketamine_response_filtered['count'])):
            # ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
            ax6.text(bar.get_x() + bar.get_width()/2, 0.1, 
                    f'n={count}', ha='center', va='bottom', fontsize=9)
    
    # 7. Forest Plot of Significant Variables
    ax7 = fig.add_subplot(gs[2, :])
    
    # Get significant variables (p < 0.05)
    sig_vars = vas_multivariate_results[vas_multivariate_results['P_value'] < 0.05].copy()
    
    if len(sig_vars) > 0:
        # Variable labels for display
        var_labels = {
            'any_opiate_dose': 'Opiate Dose (mcg)',
            'any_ketamine_dose': 'Ketamine Dose (mg)',
            'age': 'Age (years)',
            'NACA': 'NACA Score',
            'vas_scene': 'VAS at Scene',
            'mission_duration': 'Mission Duration (min)',
            'male_physician': 'Male Physician',
            'male_patient': 'Male Patient',
            'dr_male_pt_male': 'Male Dr - Male Pt',
            'dr_male_pt_female': 'Male Dr - Female Pt',
            'dr_female_pt_male': 'Female Dr - Male Pt'
        }
        
        sig_vars['display_name'] = sig_vars['Variable'].map(lambda x: var_labels.get(x, x))
        sig_vars = sig_vars.sort_values('Coefficient')
        
        y_pos = np.arange(len(sig_vars))
        
        # Plot coefficients with confidence intervals
        ax7.errorbar(sig_vars['Coefficient'], y_pos, 
                    xerr=[sig_vars['Coefficient'] - sig_vars['CI_lower'], 
                          sig_vars['CI_upper'] - sig_vars['Coefficient']], 
                    fmt='o', markersize=8, capsize=5, capthick=2)
        
        # Reference line at 0
        ax7.axvline(x=0, color='black', linestyle='--', alpha=0.5)
        
        ax7.set_yticks(y_pos)
        ax7.set_yticklabels(sig_vars['display_name'])
        ax7.set_xlabel('Beta Coefficient (95% CI)', fontweight='bold')
        ax7.set_title('Significant Predictors of VAS Reduction (Multivariate Analysis)', 
                     fontweight='bold')
        ax7.grid(True, alpha=0.3, axis='x')
        
        # Add coefficient values as text with better positioning
        for i, (coef, p_val, CI_upper) in enumerate(zip(sig_vars['Coefficient'], sig_vars['P_value'], sig_vars['CI_upper'])):
            ax7.text(CI_upper + 0.01, i, 
                    f'β={coef:.3f}\np={p_val:.3f}', 
                    va='center', fontsize=8, fontweight='bold')
    else:
        ax7.text(0.5, 0.5, 'No significant variables\n(p < 0.05)', 
                ha='center', va='center', transform=ax7.transAxes, 
                fontsize=14, fontweight='bold')
        ax7.set_xlim(0, 1)
        ax7.set_ylim(0, 1)
        ax7.set_title('Significant Predictors of VAS Reduction', fontweight='bold')
    
    plt.suptitle('VAS Reduction Analysis Summary - Adult Trauma Patients', 
                fontsize=18, fontweight='bold', y=0.96)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\\n" + "="*70)
    print("VAS REDUCTION ANALYSIS SUMMARY")
    print("="*70)
    print(f"Sample size: {len(adult_df_clean)} adult patients")
    print(f"Mean VAS reduction: {adult_df_clean['vas_reduction'].mean():.2f} ± {adult_df_clean['vas_reduction'].std():.2f}")
    print(f"Model R-squared: {vas_model_stats['R_squared']:.3f}")
    print(f"Model Adjusted R-squared: {vas_model_stats['Adj_R_squared']:.3f}")
    
    if len(sig_vars) > 0:
        print(f"\\nSignificant predictors (p < 0.05): {len(sig_vars)}")
        for _, row in sig_vars.iterrows():
            direction = "increases" if row['Coefficient'] > 0 else "decreases"
            print(f"  • {row['display_name']}: β = {row['Coefficient']:.3f} ({direction} VAS reduction)")
    else:
        print("\\nNo significant predictors identified")
    
    return fig

# Create the comprehensive summary figure
fig = create_vas_reduction_summary_figure()

In [ ]:
# save fig
# fig.savefig('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/analysis/adult_trauma/vas_reduction_summary_figure.png', dpi=300, bbox_inches='tight')